In [1]:
import numpy as np
from scipy import sparse
np.set_printoptions(formatter={'float': '{: 0.11f}'.format})

import networkx as nx
import pandas as pd
import time

import algorithms

In [12]:
n = 10
T = 1000
N_trial = 10
noise = 1

file_name = 'n{}_T{}_N{}_noise{}'.format(n, T, N_trial, noise)

# load data (random edge weights)
Wadj_TN = np.load('data/'+ file_name +'.npz')['Wadj_TN']

In [13]:
# define functions to get gradients and run experiments over instances

def get_grad(pred, popt, G, obj=None):
    grad = np.zeros(n)
    if obj == 'L-1':
        grad = -np.sign(popt - pred)
    elif obj == 'L-2sq/2':
        grad = -(popt - pred)
    elif obj == 'L-inf':
        i = np.argmax(np.abs(popt - pred))
        grad[i] = -np.sign(popt[i] - pred[i])
    elif obj == 'Lpm-inf':
        i = np.argmax(popt - pred)
        j = np.argmin(popt - pred)
        grad[i] = -1 if popt[i] > pred[i] else 0
        grad[j] = +1 if popt[j] < pred[j] else 0
    elif obj == 'mu':
        p = pred
        matching = nx.max_weight_matching(G, maxcardinality=False, weight='w')
        matching = list(map(sorted, list(matching)))

        L, R = list(range(0, int(n/2))), list(range(int(n/2), n))
        D = nx.DiGraph()
        D.add_nodes_from(L + R + [-1, n])
        for i in L + R:
            D.add_edge(-1, i, weight = 0)
            D.add_edge(i, n, weight = 0)
        for e in list(G.edges()):
            i, j = min(e[0], e[1]), max(e[0], e[1])
            D.add_edge(i, j, weight = - G[i][j]['w'] - p[j] + p[i] + 1e-10)
        for i, j in matching:
            D.add_edge(j, i, weight = G[i][j]['w'] + p[j] - p[i] + 1e-10)
        _, path = nx.single_source_bellman_ford(D, -1, n, weight='weight')
        i, j = path[1], path[-2]
        if i != j: 
            grad[i] = -1
            grad[j] = +1
    return grad

def get_graph(Wadj):
    A = sparse.csr_matrix(Wadj)
    G = nx.bipartite.from_biadjacency_matrix(A, edge_attribute='w')
    return G

def run_over_T(obj, Wadj_T, pinit = np.zeros(n), rate = 1):
    pred = np.copy(pinit)
    qvec = np.zeros(n) # for anytime online-to-batch
    num_iters = []
    Wmax = np.max(np.abs(Wadj_T))
    B = Wmax * n**1.5 # B is the radius of the L2 ball containing optimal predictions
    gsum = 0
    time_ave = 0
    for t in range(T):
        G = get_graph(Wadj_T[t])
        algs = algorithms.BipartiteMatching(G)
        if obj == 'L-1':
            feas = algs.L1_get_feasible(qvec)
        else:
            feas = algs.Lpm_get_feasible(qvec)
        res = algs.steepest_descent(feas)
        popt = res['dual']
        num_iters.append(res['num_iter'])
        
        start = time.time()
        if obj == 'cold':
            continue
        else:
            g = get_grad(qvec, popt, G, obj)
            gsum += np.linalg.norm(g)**2
            eta = rate * B*np.sqrt(2)/np.sqrt(gsum)
            pred -= rate * eta * g
            pred = np.clip(pred, -n*Wmax, n*Wmax)
            qvec = (qvec * t + pred) / (t+1)
        time_ave += time.time() - start
    return num_iters

In [ ]:
rates = np.array([.01, .1, 1, 10]) # learning rates
t_list, obj_list, rate_list, iter_list = [], [], [], []

methods = ['L-1', 'L-inf', 'mu', 'cold']
for i in range(N_trial):
    for obj in methods:
        for rate in rates:
            if obj == 'cold' and rate != rates[0]:
                continue
            pinit = np.zeros(n)
            num_iters = np.array(run_over_T(obj, Wadj_TN[i], rate=rate))
            t_list += list(range(T))
            obj_list += [obj]*T
            rate_list += [rate]*T
            iter_list += list(np.cumsum(num_iters)/np.arange(1,T+1))
            print(obj, num_iters[-1])

df = pd.DataFrame(list(zip(t_list, obj_list, rate_list, iter_list)), columns=["t", "obj", "rate", "iter"])
#df.to_pickle('result/' + file_name +'.pkl') # to save the result